In [1]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = random.choice(["안녕하세요", "사랑합니다", "배가 고파요"])
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(server_name='0.0.0.0')

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jmjung/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7868

To create a public link, set `share=True` in `launch()`.


In [1]:
import gradio as gr
import openai
import os

# OpenAI API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기
if not openai.api_key:
    raise ValueError("OpenAI API Key가 설정되지 않았습니다. 환경 변수 'OPENAI_API_KEY'를 설정하세요.")

# 답변 생성 함수
def answer(state, state_chatbot, text):
    # 이전 대화 기록에 사용자 메시지 추가
    messages = state + [{"role": "user", "content": text}]

    # ChatGPT API 호출
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # 사용 가능한 모델 이름 확인
            messages=messages
        )
        # ChatGPT의 응답 추출
        msg = response["choices"][0]["message"]["content"]

        # 새로운 대화 기록 추가
        new_state = [{"role": "user", "content": text}, {"role": "assistant", "content": msg}]
        state = state + new_state
        state_chatbot = state_chatbot + [(text, msg)]

        return state, state_chatbot, state_chatbot

    except Exception as e:
        # 오류 발생 시 응답
        error_msg = f"Error: {str(e)}"
        state_chatbot.append((text, error_msg))
        return state, state_chatbot, state_chatbot


# Gradio UI 생성
with gr.Blocks(css="#chatbot .overflow-y-auto{height:750px}") as demo:
    # 대화 상태 저장
    state = gr.State([{"role": "system", "content": "You are a helpful assistant."}])
    state_chatbot = gr.State([])

    # 헤더 UI
    with gr.Row():
        gr.HTML("""<div style="text-align: center; max-width: 500px; margin: 0 auto;">
            <div>
                <h1>Jaemin's ChatGPT</h1>
            </div>
        </div>""")

    # 채팅 UI
    with gr.Row():
        chatbot = gr.Chatbot(elem_id="chatbot")  # 채팅 인터페이스

    # 텍스트 입력 UI
    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Send a message...")  # .style() 메서드 제거

    # 이벤트 핸들링: 텍스트 제출 시
    txt.submit(answer, [state, state_chatbot, txt], [state, state_chatbot, chatbot])
    txt.submit(lambda: "", None, txt)  # 텍스트 박스 초기화

# Gradio 애플리케이션 실행
demo.launch(debug=True, share=True)


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jmjung/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7872
* Running on public URL: https://3c7dc0bc787d042909.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7872 <> https://3c7dc0bc787d042909.gradio.live


In [1]:
import os
import gradio as gr
import random
import time
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox ()
    clear = gr.ClearButton([msg, chatbot])

    def user(message, chat_history):
        chat_history.append((message, ""))
        return "", chat_history
    
    def respond(message, chat_history):
        messages = [{"role": "system", "content": "You are a helpful assistant."}]
        for user_msg, bot_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            if bot_msg:
                messages.append({"role": "assistant", "content": bot_msg})
        
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            stream=True,
        )
        
        full_response = ""
        for chunk in response:
            delta = chunk.choices[0].delta
            if hasattr(delta, "content") and delta.content:
                full_response += delta.content

        chat_history[-1] = (chat_history[-1][0], full_response)
        return "", chat_history

    clear.click(lambda: None, None, chatbot, queue=False)
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(server_name='0.0.0.0')

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jmjung/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7872

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/jmjung/.local/lib/python3.10/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "/home/jmjung/.local/lib/python3.10/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/jmjung/.local/lib/python3.10/site-packages/gradio/blocks.py", line 2042, in process_api
    result = await self.call_function(
  File "/home/jmjung/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1589, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 2461, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-pac

In [9]:
import openai
import gradio as gr

# OpenAI API 키 설정
openai.api_key =  os.getenv("OPENAI_API_KEY")

# ChatGPT와 스트리밍 대화를 처리하는 함수
def stream_chat(messages):
    """
    ChatGPT API와 통신하며 스트리밍 방식으로 응답을 반환하는 함수
    """
    try:
        # OpenAI API 호출
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=messages,
            stream=True  # 스트리밍 활성화
        )
        bot_message = ""
        # 스트리밍 응답 처리
        for chunk in response:
            if "choices" in chunk:
                delta = chunk["choices"][0]["delta"]
                if "content" in delta:
                    bot_message += delta["content"]
                    yield bot_message  # 실시간으로 업데이트
    except Exception as e:
        yield f"Error: {e}"

# Gradio에서 사용하는 함수
def chatbot_interface(user_message, chat_history):
    """
    Gradio 인터페이스에서 입력 메시지를 받아 ChatGPT와의 대화를 관리
    """
    if chat_history is None:
        chat_history = []

    # OpenAI API에 전달할 메시지 포맷 생성
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    for user, bot in chat_history:
        messages.append({"role": "user", "content": user})
        if bot is not None:
            messages.append({"role": "assistant", "content": bot})

    # 사용자 메시지를 추가
    messages.append({"role": "user", "content": user_message})
    chat_history.append([user_message, None])  # Gradio Chatbot에서 요구하는 형식

    # 스트리밍 방식으로 응답 처리
    bot_response = ""
    for chunk in stream_chat(messages):
        bot_response = chunk
        chat_history[-1][1] = bot_response  # Gradio 형식에 맞게 실시간으로 업데이트
        yield chat_history  # 실시간으로 Gradio 인터페이스에 반영

    # 최종 응답 저장
    chat_history[-1][1] = bot_response
    return chat_history

# Gradio UI 구성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="ChatGPT with Streaming")  # Chatbot 인터페이스
    user_input = gr.Textbox(label="Type your message:")  # 사용자 입력
    clear_btn = gr.Button("Clear Chat")  # 채팅 기록 초기화 버튼

    # 대화 기록을 상태로 관리
    state = gr.State([])

    # 메시지 입력 시 대화 업데이트
    user_input.submit(chatbot_interface, inputs=[user_input, state], outputs=[chatbot, state])
    clear_btn.click(lambda: ([], []), inputs=None, outputs=[chatbot, state])  # 초기화 버튼

# Gradio 애플리케이션 실행
demo.queue()
demo.launch(server_name='0.0.0.0')


/home/jmjung/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7877

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/jmjung/.local/lib/python3.10/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "/home/jmjung/.local/lib/python3.10/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/jmjung/.local/lib/python3.10/site-packages/gradio/blocks.py", line 2052, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/home/jmjung/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1808, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
  File "/home/jmjung/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1763, in validate_outputs
    raise ValueError(
ValueError: A  function (chatbot_interface) didn't return enough output values (needed: 2, returned: 1).
    Output components:
        [chatbot, state]
    Output val

In [ ]:
import gradio as gr

chat = Chat()

def add_text(state, text):
    chat.query(text)
    result = chat.messages[-1]['content']
    state = state + [(text, result)]
    return state, state


with gr.Blocks(css="#chatbot .overflow-y-auto{height:500px}") as demo:
    chatbot = gr.Chatbot(elem_id="chatbot")
    state = gr.State([])

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter, or upload an image").style(container=False)

    txt.submit(add_text, [state, txt], [state, chatbot])
    txt.submit(lambda :"", None, txt)

demo.launch(share=True, inline=True)

In [2]:
# 스트림

import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        bot_message = random.choice(["안녕하세요", "사랑합니다", "배가 고파요"])
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            time.sleep(0.05)
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)
    
demo.queue()
demo.launch(server_name='0.0.0.0')

/home/jmjung/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7869

To create a public link, set `share=True` in `launch()`.


In [3]:
import gradio as gr

def greet(history, input):
    return history + [(input, "안녕하세요 " + input + "씨")]

def vote(data: gr.LikeData):
    if data.liked:
        print("다음 출력에 대해서 좋아요를 눌렀습니다. : " + data.value)
    else:
        print("다음 출력에 대해서 싫어요를 눌렀습니다. : " + data.value)
    

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    textbox = gr.Textbox()
    textbox.submit(greet, [chatbot, textbox], [chatbot])
    chatbot.like(vote, None, None)  # Adding this line causes the like/dislike icons to appear in your chatbot
    
demo.launch(server_name='0.0.0.0')

/home/jmjung/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7870

To create a public link, set `share=True` in `launch()`.


다음 출력에 대해서 싫어요를 눌렀습니다. : 안녕하세요 안녕하세요씨
다음 출력에 대해서 좋아요를 눌렀습니다. : 안녕하세요 안녕하세요씨
다음 출력에 대해서 좋아요를 눌렀습니다. : 안녕하세요 반갑습니다.씨


In [4]:
import gradio as gr
import os
import time

def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.update(value="", interactive=False)

def add_file(history, file):
    history = history + [((file.name,), None)]
    return history

def bot(history):
    response = "멋져요!"
    history[-1][1] = ""
    for character in response:
        history[-1][1] += character
        time.sleep(0.05)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        avatar_images=(None, (os.path.join("avatar.png"))),
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="텍스트를 입력하고 엔터를 치거나 이미지를 업로드하세요",
            container=False,
        )
        btn = gr.UploadButton("Upload", file_types=["image", "video", "audio"])

    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot
    )
    txt_msg.then(lambda: gr.update(interactive=True), None, [txt], queue=False)
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.queue()
demo.launch(server_name='0.0.0.0')


/home/jmjung/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7871

To create a public link, set `share=True` in `launch()`.
